In [ ]:
!pip install -q rich

     |████████████████████████████████| 204kB 7.1MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


In [209]:
!wget https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/dev/dialogues_002.json

--2021-05-15 13:12:28--  https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/dev/dialogues_002.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10708830 (10M) [text/plain]
Saving to: ‘dialogues_002.json’

dialogues_002.json  100%[===================>]  10.21M  34.7MB/s    in 0.3s    

2021-05-15 13:12:29 (34.7 MB/s) - ‘dialogues_002.json’ saved [10708830/10708830]



In [243]:
from rich import print
import json
import copy

In [264]:
with open('/content/dialogues_001.json.2', 'r', encoding='utf-8') as f:
  data = json.load(f)

In [265]:
original_data = copy.deepcopy(data)

In [266]:
def json2txt(dialogue):
    result = []
    for turn in dialogue['turns']:
        markup = []
        for frame in turn['frames']:
            slots = filter(lambda x: "start" in x, frame['slots']) # we need only slots with start/excl_end keys
            markup += slots

        markup = sorted(markup, key=lambda x: x['start'])

        text = turn['utterance']
        offset = 0
        for slot in markup:
            pre = text[:slot['start']+offset]
            target = text[slot['start']+offset:slot['exclusive_end']+offset]
            post = text[slot['exclusive_end']+offset:]
            text = f"{pre}[{target}]{post}"
            offset += 2

        result.append(text)

    return result

In [267]:
print(json2txt(data[0]))

[
    "I'm looking for a local place to dine in the centre that serves [chinese] food.",
    'I have restaurants matching your criteria in all price ranges. Do you have a preference 
on price?',
    'I need the address, postcode and the price range.',
    'Ok how about Charlie Chan, located at Regent Street City Centre. Postcode is cb21db with
a cheap price. Can I help you further today?',
    'I also need a train. The train should leave after [16:15] and should leave on sunday.',
    "Can I have more information for the train you're needing? Where are you departing from 
and arriving to?",
    'I am leaving from Cambridge and going to Norwich.',
    'I have train TR1840 leaving at 16:36 is that okay?',
    'book for 5 people and get me the reference number',
    "You're all set. Reference number is NJB87PAP . Is there anything else I can help you 
with today?",
    'No, this is all I will need. Thank you.',
    'Thank for calling us today. I hope you have a good trip.'
]

In [268]:
len(data)

512

In [269]:
accum = ""

for x in data:
    utts = json2txt(x)
    text = "\n".join(utts)
    accum += text + "\n\n\n"

In [278]:
with open("dialogues_001.txt", 'w', encoding='utf-8') as f:
    f.write(accum)

In [279]:
with open("/content/dialogues_0001_ru.txt", 'r', encoding='utf-8') as f:
    translated = f.readlines()

In [280]:
trans = ' '.join(translated)

In [281]:
type(trans)

str

In [282]:
def parse_translate(translated_text):
    lines = translated_text.split('\n')
    texts = []
    current_text = []
    skip = False
    for line in lines:
        if not skip:
            if line != ' ':
                current_text.append(line)
            else:
                texts.append(current_text)
                skip = True
        else:
            skip = False
            current_text = []

    return texts

In [283]:
translated = parse_translate(trans)

In [284]:
len(translated)

511

In [285]:
from collections import Counter
for idx, i in enumerate(translated):
    for idx_i, t in enumerate(i):
        if Counter(t)[']'] >= 2:
            print(f"{idx}-{idx_i}: {Counter(t)[']']}")
            print(t)

2-0: 2

Мне нужно такси, чтобы приехать в [бистро на переулке] к [19:30].

23-2: 2

Я хочу, чтобы такси было здесь в [одиннадцать часов утра], чтобы я мог прибыть в пункт 
назначения [в полдень].

26-8: 2

...

[A ] - это недорогой ресторан, расположенный в центре. Отвечает ли это вашим потребностям?

496-2: 2

Я должен прибыть в [нандос] к [23:15]

508-14: 2

[мезе-бар] в порядке. забронируйте на 6 в [14:15] во вторник

**Функция, которая ищет спаны:**

In [286]:
def search_spans(text, start_symbol: str = "[", end_symbol: str = "]"):
    subtext1=start_symbol
    result1 = []
    flag = False
    n = 1
    for i,element in enumerate(text):
        if flag:
            if n > len(subtext1)-1:
                flag = False
                n = 1
            elif element == subtext1[n]:
                n += 1
            elif element != subtext1[n]:
                result1 = result1[:-1]
                n = 1
                flag = False
        if element == subtext1[0] and not flag:
            flag = True
            result1.append(i)

    subtext2=end_symbol
 
    result2 = []

    flag = False
    n = 1

    for i,element in enumerate(text):
        if flag:
            if n > len(subtext2)-1:
                flag = False
                n = 1
            elif element == subtext2[n]:
                n += 1
            elif element != subtext2[n]:
                result2 = result2[:-1]
                n = 1
                flag = False
        if element == subtext2[0] and not flag:
            flag = True
            result2.append(i)
    ress = list(zip(result1, result2))

    return ress

In [287]:
def translated2json(translated_dialogue, dialogue):
    new_dialogue = dialogue.copy()
    for i, text in enumerate(translated_dialogue):
        spans = list(reversed(search_spans(text, "[", "]"))) # reverse list, the first entry become last
        offset_front = 0
        offset_back = 1
        remove_spec_brackets = False
        for frame in new_dialogue['turns'][i]['frames']:
            for slot in frame['slots']:
                if 'start' in slot:
                    if len(spans) != 0:
                        start, end = spans.pop() # after reverse, we can use .pop() to get and remove first entry (i.e 'spans' list is now FIFO-queue)
                    elif "«" in text:
                        start, end = search_spans(text, "«", "»")[0] # assuming that there is only one span with «» brackets
                        remove_spec_brackets = True
                    else:
                        continue
                        
                    slot['start'] = start - offset_front
                    slot['exclusive_end'] = end - offset_back # offsetting '[', which will be removed
                    slot['value'] = text[start + 1:end] # offsetting not yet remove '[' for value extraction
                    offset_front += 2
                    offset_back += 2
                        

        new_text = text.replace("[", "").replace("]", "")
        if remove_spec_brackets:
              new_text = new_text.replace("«", "").replace("»", "")
        new_dialogue['turns'][i]['utterance'] = new_text
            
        #print(new_dialogue['turns'][i]['utterance'])
    return new_dialogue

In [294]:
res = translated2json(translated[6], data[6])

In [295]:
json2txt(res)

[' мне нужна информация о месте, где я могу увидеть местные достопримечательности.',
 ' Хорошо! Какую часть города вы хотели бы посетить?',
 ' Если подумать, давайте пока отложим это. Я подумывал попробовать новую кухню, пока мы в городе. Что-то дорогое.',
 ' Хорошо! Какую кухню вы бы хотели попробовать?',
 ' У меня есть пара [ямайских] приятелей, и я хочу, чтобы они чувствовали себя как дома.',
 ' У нас в городе нет дорогих ямайских заведений. Хотите попробовать что-нибудь еще?',
 ' Как насчет того, чтобы подать [африканскую] еду?',
 ' [Бедуин] в центре города - дорогой африканский ресторан. Вы хотите, чтобы я забронировал это для вас?',
 ' Мне сейчас не нужен столик, но мне нужен почтовый индекс и номер телефона этого ресторана.',
 ' номер телефона 01223367660 и почтовый индекс cb12bd',
 ' Спасибо за эту информацию. Еще мне нужны достопримечательности в том же районе города, что и ресторан. У вас есть какие-нибудь занятия на лодке?',
 ' В этом районе их 2. Это Scudamores Punting Co и

In [296]:
json2txt(data[6])

[' мне нужна информация о месте, где я могу увидеть местные достопримечательности.',
 ' Хорошо! Какую часть города вы хотели бы посетить?',
 ' Если подумать, давайте пока отложим это. Я подумывал попробовать новую кухню, пока мы в городе. Что-то дорогое.',
 ' Хорошо! Какую кухню вы бы хотели попробовать?',
 ' У меня есть пара [ямайских] приятелей, и я хочу, чтобы они чувствовали себя как дома.',
 ' У нас в городе нет дорогих ямайских заведений. Хотите попробовать что-нибудь еще?',
 ' Как насчет того, чтобы подать [африканскую] еду?',
 ' [Бедуин] в центре города - дорогой африканский ресторан. Вы хотите, чтобы я забронировал это для вас?',
 ' Мне сейчас не нужен столик, но мне нужен почтовый индекс и номер телефона этого ресторана.',
 ' номер телефона 01223367660 и почтовый индекс cb12bd',
 ' Спасибо за эту информацию. Еще мне нужны достопримечательности в том же районе города, что и ресторан. У вас есть какие-нибудь занятия на лодке?',
 ' В этом районе их 2. Это Scudamores Punting Co и

In [297]:
# generating json with translated data
translated_json = []
for i in range(len(translated)):
    res = translated2json(translated[i], data[i])
    translated_json.append(res)
    
with open("dialogues_001_russian.json", "w", encoding='utf-8') as f:
    json.dump(translated_json, f, ensure_ascii=False)

In [291]:
len(data)

512

In [292]:
len(translated)

511

In [298]:
# testing
for i in range(len(translated)):
    test_reference = translated[i] # reference version from translated texts
    res = translated2json(translated[i], data[i])
    orig = json2txt(original_data[i])
    test_target = json2txt(res)
    for j in range(len(translated[i])):
        if test_reference[j] != test_target[j]:
            print(f"Error at dialogue #{i} - utterance #{j}:")
            print(f"[bold green]Original[/bold green]: {orig[j]}")
            print(f"[bold yellow]Translation[/bold yellow]: {test_reference[j]}")
            print(f"[bold red]After matching[/bold red]: {test_target[j]}")
            print("=="*20)
    #print(test_reference)
    #print("="*20)
    #print(test_target)
    #assert test_reference == test_target, f"Failed for text {i}"

Error at dialogue #6 - utterance #7:

Original: The [Bedouin] in the city centre is an expensive African restaurant. Would you like
me to book it for you?

Translation:  «Бедуин» в центре города - дорогой африканский ресторан. Вы хотите, чтобы я 
забронировал это для вас?

After matching:  [Бедуин] в центре города - дорогой африканский ресторан. Вы хотите, чтобы я 
забронировал это для вас?

========================================

...

Error at dialogue #494 - utterance #1:

Original: [The A] is a moderate priced restaurant located in the centre. Would that meet your
needs?

Translation:  [A ] - это недорогой ресторан, расположенный в центре. Отвечает ли это вашим 
потребностям?

After matching:  [A ]- это недорогой ресторан, расположенный в центре. Отвечает ли это вашим 
потребностям?

========================================